# 10.1 GroupBy 메카닉

In [2]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [3]:
df = pd.DataFrame({
    'key1' : ['a', 'a', 'b', 'b', 'a'],
    'key2' : ['one', 'two', 'one', 'two', 'one'],
    'data1' : np.random.randn(5),
    'data2' : np.random.randn(5)
})

df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


### 각 그룹에서 data1의 평균 구하기

![img](./img/groupby.jfif)

In [13]:
# Groupby - Single Group Single Column
grouped = df['data1'].groupby(df['key1'])
grouped

컬럼 이름으로 색인 했을 경우 **SeriesGroupBy**가 나오는 것을 확인할 수 있다.  
이에 대한 추가적인 내용은 뒤에서 다룰 예정이다.

집단별 크기는 grouped.size(), 집단별 합계는 grouped.sum(), 집단별 평균은 grouped.mean() 을 사용한다.

In [17]:
# 크기
grouped.size()

key1
a    3
b    2
Name: data1, dtype: int64

In [18]:
# 합계
grouped.sum()

key1
a    2.240016
b   -1.075169
Name: data1, dtype: float64

In [19]:
# 평균
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

In [15]:
# Groupby - Single Group Single Column
#df['data1'].groupby(df['key1']).mean() # 1
df.groupby(['key1'])['data1'].mean() # 2

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

위에서 처럼 바로 Apply 함수를 적용하는 것도 가능하다.  
또한 1번처럼 사용해도 되고 2번처럼 사용할 수 있다.  
1번과 2번은 다음과 같이 해석 할 수 있다. 
<br>
1. df['data1']을 df['key1'] 그룹을 기준으로 집계한다.  
2. df의 ['key1'] 그룹을 기준으로 data1을 집계한다.   

In [26]:
# Groupby - Multiple Columns
means = df['data1'].groupby([df['key1'], df['key2']]).mean() # 1
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

위의 예처럼 여러 개의 범주형 변수 key 값을 가지고 그룹별 집계를 수행할 수도 있다.  하지만 여기서도 마찬가지로 df가 많이 쓰여서 가독성이 떨어진다.  
그래서 다음과 같이 코드를 깔끔하게 변형할 수 있다.

In [31]:
# 1번 코드의 비해 가독성이 더 좋다.
df.groupby(['key1', 'key2'])['data1'].mean()

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [23]:
type(means)

pandas.core.series.Series

집계 결과가 Series 형태로 표현되는 것을 확인할 수 있다.  
이때 unstack() 함수를 사용하면 집계 결과를 가로, 세로 축으로 좀더 보기 좋게 표현 할 수 있다.

In [9]:
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


In [32]:
type(means.unstack())

pandas.core.frame.DataFrame

data1에 괄호를 하나 더 추가해도 DataFrame 형식으로 나오는 것을 확인 할 수 있다.

In [33]:
# df.groupby(['key1', 'key2'])['data1'].mean()
df.groupby(['key1', 'key2'])[['data1']].mean()

data1
key1 key2          
a    one   0.880536
     two   0.478943
b    one  -0.519439
     two  -0.555730

이는 groupby 객체를 컬럼 이름으로 색인 했을 때와 컬럼 이름이 담긴 배열로 색인 했을 때의 차이가 있다.  
https://steadiness-193.tistory.com/123 자세한 내용은 이 링크에 담겨져 있다.

In [34]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [39]:
group = df.groupby('key1')
group

컬럼 이름이 담긴 배열로 색인한 경우 **DataFrameGroupBy**가 나온다.  
여기서 컬럼 이름이 담긴 배열로 인식하는 이유는 명시적으로 'data1'을 색인하지 않았기 때문에 ['data1', 'data2']가 담긴 배열로 자동 지정이 되기 때문이다.

In [38]:
group.mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


결과도 마찬가지로 데이터프레임 형식으로 나오게 된다.

In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

In [12]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 그룹별 반복 작업

In [6]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023


In [7]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023


In [8]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


In [9]:
df.dtypes
grouped = df.groupby(df.dtypes, axis=1)

In [10]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
